<font size="3"><b>Section 1: Common data sources:</b></font><br>
        <font size="3">1. Navigate to www.kaggle.com and look for a used cars data set. What's the content?</font><br>
        <font size="3">2. Download an SQLite-Database from www.kaggle.com. Can you make an SQL request on the DB?</font><br>
        <font size="3">3. Navigate to www.autoscout24.ch. Can you derive the xpath of car prices from the website?</font><br>

Import

In [1]:

import sqlite3
import fnmatch
import pandas as pd

Turn off warnings

In [2]:
import warnings
# Ignore warnings
warnings.filterwarnings('ignore')

Connect to database

In [3]:
# Create a connection to the database. Source: https://www.kaggle.com/datasets/guillemservera/tennis
conn = sqlite3.connect('tennisplayers.sqlite')
# Create a cursor
cursor = conn.cursor()

Show tables in database

In [5]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
# Print the table names
for name in res:
    print(name[0])

matches
players
rankings


Get colums of players table

In [9]:
cursor.execute('''PRAGMA table_info(players)''')
for col in cursor.fetchall():
    print(col)

(0, 'player_id', 'INTEGER', 0, None, 0)
(1, 'name_first', 'TEXT', 0, None, 0)
(2, 'name_last', 'TEXT', 0, None, 0)
(3, 'hand', 'TEXT', 0, None, 0)
(4, 'dob', 'REAL', 0, None, 0)
(5, 'ioc', 'TEXT', 0, None, 0)
(6, 'height', 'REAL', 0, None, 0)
(7, 'wikidata_id', 'TEXT', 0, None, 0)


Query database

In [14]:
cursor.execute('''SELECT * FROM players
                  WHERE name_last LIKE "A%"''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['player_id','name_first','name_last'])    
df

ValueError: 3 columns passed, passed data had 8 columns

Get Autoscout webpage

In [15]:
import requests
from bs4 import BeautifulSoup

# Request a websites content
page = requests.get("https://www.autoscout24.ch/de/autos/alle-marken?vehtyp=10")

# Parsing the page content with BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# Print the page content
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <noscript data-next-hide-fouc="true">
   <style>
    body{display:block}
   </style>
  </noscript>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="data:image/x-icon;base64,AAABAAEAAAAAAAEAIACwFgAAFgAAAIlQTkcNChoKAAAADUlIRFIAAAEAAAABAAgGAAAAXHKoZgAA
    AAFvck5UAc+id5oAABZqSURBVHja7Z1nfFTltodDL4HQQu8llJCQSaEqTZBeFERQEBUEpaiASjly
    EEUFERUQVA6i2BUpKoiCHQvqEQUERZp0pNivlXtdd681mWQSEsRzZM9k9vPh+RlpM5ns9V/1XW9U
    amqqAIA3ieJDAEAAAAABAAAEAAAQAABAAAAAAQAABAAAEAAAQAAAAAEAAAQAABAAAEAAAAABAAAE
    AAAQAABAAAAAAQAABAAAEAAAQAAAAAEAAAQAABAAAEAAAAABAAAEAAAQAABAAAAAAQAABAAAEAAA
    QAAAAAEAAAQAABAAAASADwEAAQAABAAAEAAAQAAAAAEAAAQAABAAAEAAAAABAAAEAAAQAABAAAAA
    AQAABAAAEAAAQAAAAAEAAAQAABAAAEAAAAABAAAEAAAQAABAAAAAAQAABAAAEAAAQAAAAAEAAAQA
    ABAAgNCQkpIqyclp4nNo2jQFAQDwisE3a5YiXTo1keGX15e7b6sm119TR9LSEACAiEINXklzvu7Q
    PkkuHdhQZt5cXV58uqx89FpJeWVZGZl/ZxXpf0G8dOrYRJo3T0EAAPKkhzeDTxWfL83+/+yzkuXC
    vvFy4/W1Z

XPath Selector

In [16]:
# XPath and CSS-selectors are helpful for the selection of text-elements from HTML-pages
# -> https://data-lessons.github.io/library-webscraping-DEPRECATED/04-lxml/
# -> https://www.freeformatter.com/xpath-tester.html

import requests
import lxml.html

# Example page from above
response = requests.get('https://www.autoscout24.ch/de/autos/alle-marken?vehtyp=10')
tree = lxml.html.fromstring(response.text)
title_elem = tree.xpath('//p')[0]
title_elem = tree.cssselect('p')[0]  # equivalent to previous XPath

print("tag:", title_elem.tag)
print("text:", title_elem.text_content())
print("html:", lxml.html.tostring(title_elem))
print("parent's tag:", title_elem.getparent().tag)

ImportError: cssselect does not seem to be installed. See https://pypi.org/project/cssselect/